# Training a Classifier on the *Salammbô* Dataset with PyTorch
Author: Pierre Nugues

We first need to import some modules

In [26]:
import torch
import torch.nn as nn
import numpy as np

### Reading the dataset
We can read the data from a file with the svmlight format or directly create numpy arrays

In [27]:
X = np.array(
    [[35680, 2217], [42514, 2761], [15162, 990], [35298, 2274],
     [29800, 1865], [40255, 2606], [74532, 4805], [37464, 2396],
     [31030, 1993], [24843, 1627], [36172, 2375], [39552, 2560],
     [72545, 4597], [75352, 4871], [18031, 1119], [36961, 2503],
     [43621, 2992], [15694, 1042], [36231, 2487], [29945, 2014],
     [40588, 2805], [75255, 5062], [37709, 2643], [30899, 2126],
     [25486, 1784], [37497, 2641], [40398, 2766], [74105, 5047],
     [76725, 5312], [18317, 1215]
     ])

y = np.array(
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Scaling the Data
Scaling and normalizing are usually very significant with neural networks. We use sklean transformers. They consist of two main methods: `fit()` and `transform()`.

### Normalizing

In [28]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X)
X_norm = normalizer.transform(X)
X_norm[:4]

array([[0.99807515, 0.06201605],
       [0.99789783, 0.06480679],
       [0.99787509, 0.06515607],
       [0.99793128, 0.06428964]])

### Standardizing

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(X_norm)
X_scaled = scaler.transform(X_norm)
X_scaled[:4]

array([[ 1.68336574, -1.7197772 ],
       [ 0.57376529, -0.56145427],
       [ 0.43143908, -0.41648279],
       [ 0.78308579, -0.77610221]])

## Creating PyTorch Tensors
PyTorch has its own implementation of matrices called tensors. They are more or less equivalent to NumPy arrays. We need to convert our dataset to these tensors and represent $\mathbf{y}$  as a column vector

In [30]:
Y = y.reshape((-1, 1))
Y[:4]

array([[0],
       [0],
       [0],
       [0]])

In [31]:
X_scaled = torch.Tensor(X_scaled)
Y = torch.Tensor(Y)
Y[:4]

tensor([[0.],
        [0.],
        [0.],
        [0.]])

## Creating a Model

We set a seed to have reproducible results

In [32]:
np.random.seed(1337)

We define a classifier equivalent to a logistic regression

In [33]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 1)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        return x

And a model with one hidden layer

In [34]:
class Model2(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 10)
        self.fc2 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

We create the model. To try the network with one hidden layer, set `complex` to true

In [35]:
complex = True

In [36]:
input_dim = X_scaled.shape[1]
if not complex:
    model = Model(input_dim)
else:
    model = Model2(input_dim)
loss_fn = nn.BCELoss()    # binary cross entropy loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

## Fitting the Model
We will show three methods: batch gradient descent, stochastic descent, and minibatches

### Batch Gradient Descent

We fit the whole dataset (batch gradient descent)

In [37]:
model.train()               # sets PyTorch in the train mode
for epoch in range(100):
    Y_pred = model(X_scaled)
    loss = loss_fn(Y_pred, Y)
    optimizer.zero_grad()   # resets the gradients
    loss.backward()         # gradient backpropagation
    optimizer.step()        # weight updates

### Stochastic Gradient Descent

or, we fit the model with a batch size of one item (stochastic gradient descent)

In [38]:
model.train()
for epoch in range(50):
    for x_scaled, y in zip(X_scaled, Y):
        y_pred = model(x_scaled)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

### Minibatch Gradient Descent

Or we fit it with mini-batches, first with a simple inner loop

In [39]:
batch_size = 4
model.train()
for epoch in range(50):
    # Would need to shuffle X and y
    for i in range(0, X_scaled.size()[0], batch_size):
        Y_batch_pred = model(X_scaled[i:i + batch_size])
        loss = loss_fn(Y_batch_pred, Y[i:i + batch_size])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Then with a dataloader

In [40]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_scaled, Y)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [41]:
model.train()
for epoch in range(50):
    for X_scaled_batch, Y_batch in dataloader:
        Y_batch_pred = model(X_scaled_batch)
        loss = loss_fn(Y_batch_pred, Y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

## The weights

In [42]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.9925,  0.9792],
         [ 0.5967,  0.4295],
         [-0.0740, -0.4229],
         [ 0.4873, -0.3297],
         [ 0.8646,  0.2097],
         [ 0.2680, -0.0328],
         [ 0.8847, -1.0460],
         [-0.6897,  0.5722],
         [ 0.7204,  0.0800],
         [ 0.6851, -0.9743]], requires_grad=True),
 Parameter containing:
 tensor([ 0.9255, -0.5765, -0.5019, -0.6609,  0.5870, -0.5801,  0.1476,  0.7072,
          0.1780,  0.3864], requires_grad=True),
 Parameter containing:
 tensor([[ 1.5852, -0.1186,  0.2697,  0.1481, -0.4031, -0.2504, -1.0789,  1.0288,
          -0.4112, -1.1202]], requires_grad=True),
 Parameter containing:
 tensor([-0.1440], requires_grad=True)]

Also in the form of a dictionary

In [43]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[-0.9925,  0.9792],
                      [ 0.5967,  0.4295],
                      [-0.0740, -0.4229],
                      [ 0.4873, -0.3297],
                      [ 0.8646,  0.2097],
                      [ 0.2680, -0.0328],
                      [ 0.8847, -1.0460],
                      [-0.6897,  0.5722],
                      [ 0.7204,  0.0800],
                      [ 0.6851, -0.9743]])),
             ('fc1.bias',
              tensor([ 0.9255, -0.5765, -0.5019, -0.6609,  0.5870, -0.5801,  0.1476,  0.7072,
                       0.1780,  0.3864])),
             ('fc2.weight',
              tensor([[ 1.5852, -0.1186,  0.2697,  0.1481, -0.4031, -0.2504, -1.0789,  1.0288,
                       -0.4112, -1.1202]])),
             ('fc2.bias', tensor([-0.1440]))])

## Prediction

We compute the probabilities to belong to class 1 for all the training set

In [44]:
model.eval()
y_pred_proba = model(X_scaled)
y_pred_proba[:4]

tensor([[0.0002],
        [0.0270],
        [0.0676],
        [0.0107]], grad_fn=<SliceBackward0>)

We recompute it with matrices

In [45]:
m_params = list(model.parameters())

In [46]:
if complex:
    print(torch.sigmoid(torch.relu(X_scaled @ m_params[0].T + m_params[1]) @ m_params[2].T + m_params[3])[:4])
else:
    print(torch.sigmoid(X_scaled @ m_params[0].T + m_params[1])[:4])

tensor([[0.0002],
        [0.0270],
        [0.0676],
        [0.0107]], grad_fn=<SliceBackward0>)


## Evaluation

In [47]:
from sklearn.metrics import accuracy_score

In [48]:
def predict_class(preds):
    c = []
    for x in range(len(preds)):
        if(preds[x] >= 0.5):
            c += [1]
        else:
            c += [0]
    return np.array(c)

In [49]:
y_pred = predict_class(y_pred_proba)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [50]:
accuracy_score(Y, y_pred)

1.0

We computed the accuracy from the training set. This is not a good practice. We should use a dedicated test set instead.